In [ ]:
import cv2
import numpy as np
from tensorflow import keras
import RPi.GPIO as GPIO
from IPython.display import display, Image
from PIL import Image as PILImage

# GPIO setup for LED
led_pin = 18
GPIO.setmode(GPIO.BCM)
GPIO.setup(led_pin, GPIO.OUT)

# Load the pre-trained models
classification_model = keras.models.load_model("Classification.h5")
segmentation_model = keras.models.load_model("DiseasedPlantFCN.h5")

# Function to classify if a plant is diseased or not
def classify_disease(image):
    # Preprocess the image for classification
    image = cv2.resize(image, (224, 224))
    image = image / 255.0
    image = np.expand_dims(image, axis=0)

    # Make predictions using the classification model
    prediction = classification_model.predict(image)

    # Assuming your model outputs binary classification
    is_diseased = (prediction > 0.5).astype(int)[0][0]
    return is_diseased

# Function to perform segmentation on a diseased image
def segment_diseased(image):
    # Preprocess the image for segmentation
    image = cv2.resize(image, (224, 224))
    image = image / 255.0
    image = np.expand_dims(image, axis=0)

    # Make predictions using the segmentation model
    prediction = segmentation_model.predict(image)

    # Assuming your model outputs binary segmentation
    segmented_image = (prediction > 0.5).astype(np.uint8)[0, :, :, 0]
    return segmented_image

# Function to capture video frames and perform classification and segmentation
def process_video():
    cap = cv2.VideoCapture(0)  # Adjust camera index if needed
    while True:
        ret, frame = cap.read()

        # Classify if the plant is diseased
        is_diseased = classify_disease(frame)

        if is_diseased:
            # Turn on the LED
            GPIO.output(led_pin, GPIO.HIGH)

            # Perform segmentation
            segmented_image = segment_diseased(frame)

            # Display the segmented image
            display(PILImage.fromarray(segmented_image * 255, 'L'))
        else:
            # Turn off the LED
            GPIO.output(led_pin, GPIO.LOW)

        # Display the original frame
        display(Image(data=cv2.imencode('.jpg', frame)[1].tobytes()))

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    GPIO.cleanup()

# Start processing video
process_video()
